# Summary of images processed

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from file_checks import *

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n raw_dir = {} \n smooth_dir = {} \n home_dir = {}"
      .format(basic_config['raw_dir'],basic_config['smooth_dir'],basic_config['home_dir']))

In [ ]:
###To get all files processed in brdf directory across all processed cells:
all_files = print_files_in_multiple_directories(basic_config['raw_dir'],"brdf",'.nc',print_list=basic_config['print_list'],out_dir=basic_config['home_dir'])

In [ ]:
#all_images.groupby(['yr','sensor']).size().unstack().plot(kind='bar', stacked=True, figsize=(20, 5), title=('Images per year'))
all_files.groupby(['yr','sensor']).size().unstack().plot(kind='bar', stacked=True, figsize=(20, 5),
            title=('Images processed for {}'.format(basic_config['country'])));

## To check all processed cells for missing files at download:

In [ ]:
dl_db_path = "/home/downspout-cel/paraguay_lc/cell_processing_dl.csv"
dl_db = pd.read_csv(Path(dl_db_path),index_col=[0])
dl_fix = dl_db[(dl_db['dl_fix_now']!='[]') & (pd.notnull(dl_db['dl_fix_now']))]
print(dl_fix)

## Get full processing status
(note, there are some methods in file_checks.py to get status of single and all cells. Most were written before this db was created. Need to consolidate & finish processing checks.

In [ ]:
full_db_path = "/home/downspout-cel/paraguay_lc/cell_processing_post.csv"
post_db = pd.read_csv(Path(full_db_path),index_col=[0])
post_db.head(n=5)

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html

out_name = str(basic_config['country']+'5a_ImagesProcessed')
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$out_name 5a_SummarizeData_ImagesProcessed.ipynb